In [1]:
import simtk.unit as unit
from simtk.openmm.app import AmberPrmtopFile, AmberInpcrdFile, PME, HBonds
from simtk.openmm import CustomNonbondedForce, NonbondedForce, XmlSerializer

## Import Amber Files

In [2]:
prmtop = AmberPrmtopFile("CB7-AMT-sol.prmtop")
inpcrd = AmberInpcrdFile("CB7-AMT-sol.rst7")

system = prmtop.createSystem(
    nonbondedMethod=PME,
    nonbondedCutoff=9.0 * unit.angstrom,
    constraints=HBonds,
    rigidWater=True,
)

with open("system_normal.xml", "w") as f:
    f.write(XmlSerializer.serialize(system))

nonbonded = [force for force in system.getForces() if isinstance(force, NonbondedForce)][0]

## Get atom indices of molecules in system

In [3]:
guest = [atom.index for atom in prmtop.topology.atoms() if atom.residue.name == "AMT"]
host_and_solvent = [atom.index for atom in prmtop.topology.atoms() if atom.residue.name != "AMT"]

## Create Weeks-Chandler-Anderson perturbation
http://www.sklogwiki.org/SklogWiki/index.php/Weeks-Chandler-Andersen_perturbation_theory

### Repulsive perturbation

$$
\Phi_{repulsive}(r) = \left\{
    \begin{array}\\
        \Phi_{LJ} + \epsilon & \mbox{if } \ r < 2^{1/6}\sigma \\
        0 & \mbox{if } \ r \ge 2^{1/6}\sigma \\
    \end{array}
\right.
$$

In [4]:
wca_repulsive = CustomNonbondedForce(
    "step(2^(1/6)*sigma - r) * (LJ + epsilon);"
    "LJ=4*epsilon*((sigma/r)^12-(sigma/r)^6);"
    "sigma=0.5*(sigma1+sigma2);"
    "epsilon=sqrt(epsilon1*epsilon2)"
)
wca_repulsive.addPerParticleParameter("sigma")
wca_repulsive.addPerParticleParameter("epsilon")
wca_repulsive.setNonbondedMethod(CustomNonbondedForce.CutoffPeriodic)
wca_repulsive.setCutoffDistance(nonbonded.getCutoffDistance())
wca_repulsive.setUseLongRangeCorrection(nonbonded.getUseDispersionCorrection())
wca_repulsive.addInteractionGroup(guest, host_and_solvent)

# Set LJ parameters
for atom in range(nonbonded.getNumParticles()):
    wca_repulsive.addParticle(nonbonded.getParticleParameters(atom)[1:])
    
system.addForce(wca_repulsive)

5

### Attractive perturbation


$$
\Phi_{attractive}(r) = \left\{
    \begin{array}\\
        -\epsilon & \mbox{if } \ r < 2^{1/6}\sigma \\
        \Phi_{LJ} & \mbox{if } \ r \ge 2^{1/6}\sigma \\
    \end{array}
\right.
$$

In [5]:
wca_attractive = CustomNonbondedForce(
    "step(2^(1/6)*sigma - r)*(-1*epsilon) + step(r - 2^(1/6)*sigma)*LJ;"
    "LJ=4*epsilon*((sigma/r)^12-(sigma/r)^6);"
    "sigma=0.5*(sigma1+sigma2);"
    "epsilon=sqrt(epsilon1*epsilon2)"
)
wca_attractive.addPerParticleParameter("sigma")
wca_attractive.addPerParticleParameter("epsilon")
wca_attractive.setNonbondedMethod(CustomNonbondedForce.CutoffPeriodic)
wca_attractive.setCutoffDistance(nonbonded.getCutoffDistance())
wca_attractive.setUseLongRangeCorrection(nonbonded.getUseDispersionCorrection())
wca_attractive.addInteractionGroup(guest, host_and_solvent)

# Set LJ parameters
for atom in range(nonbonded.getNumParticles()):
    wca_attractive.addParticle(nonbonded.getParticleParameters(atom)[1:])
    
system.addForce(wca_attractive)

6

## Set Guest molecule LJ parameters to zero

In [6]:
for atom in AdOH:
    current_params = nonbonded.getParticleParameters(atom)
    new_params = [current_params[0]*0.0, current_params[1]*0.0, current_params[2]*0.0]
    nonbonded.setParticleParameters(atom, *new_params)

## Save new system

In [7]:
with open("system_wca.xml", "w") as f:
    f.write(XmlSerializer.serialize(system))